In [161]:
#Generate new random orders for a pantry for past n years

import csv
import random
import datetime
import pandas as pd

# Constants for food size ranges
SIZE_RANGES = {
    "Small": (4000, 6000),
    "Medium": (6000, 8000),
    "Large": (8000, 9000)
}

# read the data from the excel file
df = pd.read_excel("DataModel.xlsx", sheet_name="Pantry")

# Group the pantries by ID and size
pantries = {}
for idx, row in df.iterrows():
    if row['PantryName'] != 'PantryName':
        pantry_id = row['PantryID']
        size = row['Size']
        if pantry_id not in pantries:
            pantries[pantry_id] = {}
        if size not in pantries[pantry_id]:
            pantries[pantry_id][size] = []
        pantries[pantry_id][size].append(row.to_dict())

# Generate food requests for each pantry and size
food_reqs = []
for pantry_id in pantries:
    for size in pantries[pantry_id]:
        rows = pantries[pantry_id][size]
        for i in range(random.randint(50, 80)):
            order_date = pd.to_datetime('now') - pd.Timedelta(days=random.randint(1, 730))
            delivery_date = order_date + pd.Timedelta(days=random.randint(4, 7))
            delivery_time = (pd.Timestamp.today().replace(hour=9, minute=0, second=0) + pd.Timedelta(hours=random.randint(0, 9))).strftime('%I:%M %p')
            order_date = order_date.strftime('%m-%d-%Y')
            delivery_date = delivery_date.strftime('%m-%d-%Y')
            quantity = random.randint(SIZE_RANGES[size][0], SIZE_RANGES[size][1])
            random_number = random.randint(100000, 900000)
            orderId = 'AFCBO' + str(random_number)
            food_reqs.append({'OrderId': orderId, 'PantryID': pantry_id, 'Order_Date': order_date, 'Delivery_Date': delivery_date, 'Delivery_Time': delivery_time, 'Quantity': quantity})

# Create a pandas DataFrame from the food requests
df_food_reqs = pd.DataFrame(food_reqs)

with pd.ExcelWriter("DataModel.xlsx", mode="a") as writer:
    df_food_reqs.to_excel(writer, sheet_name="FoodOrders", index=False)


<ipython-input-161-896ce41a5635>:34: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  order_date = pd.to_datetime('now') - pd.Timedelta(days=random.randint(1, 730))


In [165]:
#Add list of random items to the order basket for all orders

import pandas as pd
import random

# Read the input Excel file and select the FoodOrders2 sheet
df = pd.read_excel('DataModel.xlsx', sheet_name='FoodOrders')

# Choose random items from the list of 50 and concatenate them
items = []
for i in range(df.shape[0]):
    items = []
    for j in range(random.randint(20, 50)):
        items.append(random.choice([
            'Canned vegetables',
            'Canned fruits',
            'Canned beans',
            'Rice',
            'Pasta',
            'Canned soup',
            'Cereal',
            'Oatmeal',
            'Peanut butter',
            'Jelly or jam',
            'Canned tuna or chicken',
            'Canned chili',
            'Tomato sauce',
            'Macaroni and cheese',
            'Boxed potatoes',
            'Boxed stuffing mix',
            'Boxed cake mix',
            'Granola bars',
            'Crackers',
            'Chips',
            'Pretzels',
            'Popcorn',
            'Trail mix',
            'Nuts',
            'Dried fruit',
            'Fruit cups',
            'Pudding cups',
            'Jello cups',
            'Shelf-stable milk',
            'Evaporated milk',
            'Condensed milk',
            'Instant coffee',
            'Tea bags',
            'Hot cocoa mix',
            'Sugar',
            'Flour',
            'Baking powder',
            'Baking soda',
            'Vanilla extract',
            'Cooking oil',
            'Vinegar',
            'Ketchup',
            'Mustard',
            'Mayonnaise',
            'Salad dressing',
            'Spices',
            'Condiments',
            'Canned spaghetti or ravioli',
            'Instant mashed potatoes',
            'Pancake mix and syrup'
        ]))
    items = list(set(items))   
    item_str = ', '.join(items)

    # Add a column to the DataFrame with the items
    df.loc[i, 'Items'] = item_str

# Write the updated DataFrame to a new sheet in the Excel file
with pd.ExcelWriter('DataModel.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='FoodOrders', index=False)


In [254]:
#Find the most popular basket for an order for all Pantries

import pandas as pd

# read the data from the excel file
df = pd.read_excel("DataModel.xlsx", sheet_name="FoodOrders")

# split the items column into a list of items for each row
df["Items"] = df["Items"].str.split(", ")

# create a new dataframe to store the top 15 items for each pantry
top_items_df = pd.DataFrame(columns=["PantryId", "TopItems"])

# loop over each unique pantry ID
for pantry_id in df["PantryID"].unique():
    
    # filter the dataframe to only include rows with the current pantry ID
    pantry_df = df[df["PantryID"] == pantry_id]
    
    # create a dictionary to store the frequency count of each item for the current pantry
    item_counts = {}
    
    # loop over each row in the filtered dataframe
    for index, row in pantry_df.iterrows():
        #print(row["Items"])
        # loop over each item in the current row
        for item in row["Items"]:
            
            # add the item to the dictionary if it doesn't exist, and increment the count if it does
            if item not in item_counts:
                item_counts[item] = 1
            else:
                item_counts[item] += 1
    
    # sort the items by their frequency count in descending order
    sorted_items = sorted(item_counts.items(), key=lambda x: x[1], reverse=True)
    
    # take the top 15 items and store them in a list
    top_items = [item[0] for item in sorted_items[:15]]

    # Convert the list of top items to a comma-separated string
    top_items_str = ', '.join([x for x in top_items])
    
    # add the pantry ID and top 15 items to the new dataframe
    top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)

# save the top items dataframe to a new sheet in the same excel file
with pd.ExcelWriter("DataModel.xlsx", mode="a") as writer:
    top_items_df.to_excel(writer, sheet_name="TopItems", index=False)

['Mayonnaise', 'Condensed milk', 'Boxed stuffing mix', 'Granola bars', 'Canned chili', 'Crackers', 'Ketchup', 'Evaporated milk', 'Dried fruit', 'Canned beans', 'Vanilla extract', 'Cooking oil', 'Jelly or jam', 'Oatmeal', 'Canned vegetables', 'Boxed potatoes', 'Baking soda', 'Jello cups', 'Instant coffee', 'Baking powder', 'Mustard', 'Vinegar', 'Nuts', 'Trail mix', 'Macaroni and cheese', 'Shelf-stable milk', 'Rice', 'Hot cocoa mix', 'Salad dressing']
['Mayonnaise', 'Pudding cups', 'Pasta', 'Popcorn', 'Crackers', 'Evaporated milk', 'Dried fruit', 'Canned beans', 'Vanilla extract', 'Jelly or jam', 'Cereal', 'Canned tuna or chicken', 'Sugar', 'Pancake mix and syrup', 'Flour', 'Jello cups', 'Instant mashed potatoes', 'Instant coffee', 'Baking powder', 'Vinegar', 'Nuts', 'Trail mix', 'Boxed cake mix', 'Tomato sauce', 'Pretzels', 'Condiments', 'Canned fruits', 'Salad dressing']
['Mayonnaise', 'Condensed milk', 'Fruit cups', 'Boxed stuffing mix', 'Pasta', 'Canned chili', 'Popcorn', 'Crackers',

<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

['Mayonnaise', 'Canned chili', 'Jello cups', 'Shelf-stable milk', 'Ketchup', 'Instant coffee', 'Instant mashed potatoes', 'Tomato sauce', 'Baking powder', 'Canned spaghetti or ravioli', 'Hot cocoa mix', 'Mustard', 'Cooking oil', 'Jelly or jam', 'Canned fruits', 'Canned vegetables']
['Mayonnaise', 'Condensed milk', 'Boxed stuffing mix', 'Granola bars', 'Pasta', 'Popcorn', 'Peanut butter', 'Evaporated milk', 'Canned beans', 'Vanilla extract', 'Oatmeal', 'Spices', 'Canned vegetables', 'Boxed potatoes', 'Canned tuna or chicken', 'Flour', 'Instant coffee', 'Mustard', 'Nuts', 'Chips', 'Canned soup', 'Canned spaghetti or ravioli', 'Pretzels', 'Rice', 'Hot cocoa mix', 'Canned fruits']
['Mayonnaise', 'Condensed milk', 'Fruit cups', 'Boxed stuffing mix', 'Pudding cups', 'Tea bags', 'Popcorn', 'Crackers', 'Peanut butter', 'Evaporated milk', 'Canned beans', 'Cooking oil', 'Spices', 'Canned vegetables', 'Boxed potatoes', 'Canned tuna or chicken', 'Sugar', 'Baking soda', 'Flour', 'Jello cups', 'Inst

<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

['Condensed milk', 'Pudding cups', 'Granola bars', 'Tea bags', 'Crackers', 'Ketchup', 'Peanut butter', 'Evaporated milk', 'Dried fruit', 'Canned beans', 'Cooking oil', 'Jelly or jam', 'Oatmeal', 'Spices', 'Cereal', 'Baking soda', 'Canned tuna or chicken', 'Sugar', 'Pancake mix and syrup', 'Flour', 'Jello cups', 'Instant mashed potatoes', 'Instant coffee', 'Baking powder', 'Mustard', 'Canned soup', 'Macaroni and cheese', 'Boxed cake mix', 'Shelf-stable milk', 'Tomato sauce', 'Rice', 'Pretzels', 'Hot cocoa mix', 'Condiments', 'Salad dressing']
['Mayonnaise', 'Condensed milk', 'Pudding cups', 'Boxed stuffing mix', 'Granola bars', 'Tea bags', 'Crackers', 'Ketchup', 'Peanut butter', 'Evaporated milk', 'Dried fruit', 'Vanilla extract', 'Oatmeal', 'Spices', 'Cereal', 'Boxed potatoes', 'Baking soda', 'Sugar', 'Instant mashed potatoes', 'Instant coffee', 'Vinegar', 'Chips', 'Trail mix', 'Canned soup', 'Boxed cake mix', 'Tomato sauce', 'Rice', 'Pretzels', 'Condiments', 'Canned fruits']
['Condens

...

<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

['Baking soda', 'Popcorn', 'Mayonnaise', 'Canned soup', 'Condensed milk', 'Crackers', 'Fruit cups', 'Pudding cups', 'Ketchup', 'Instant coffee', 'Shelf-stable milk', 'Canned spaghetti or ravioli', 'Rice', 'Canned fruits', 'Salad dressing', 'Vinegar']
['Mayonnaise', 'Condensed milk', 'Fruit cups', 'Pudding cups', 'Boxed stuffing mix', 'Pasta', 'Tea bags', 'Popcorn', 'Peanut butter', 'Evaporated milk', 'Dried fruit', 'Canned beans', 'Vanilla extract', 'Cooking oil', 'Oatmeal', 'Canned vegetables', 'Boxed potatoes', 'Canned tuna or chicken', 'Baking soda', 'Jello cups', 'Mustard', 'Nuts', 'Chips', 'Canned soup', 'Boxed cake mix', 'Shelf-stable milk', 'Canned spaghetti or ravioli', 'Tomato sauce', 'Rice', 'Condiments', 'Canned fruits']
['Mayonnaise', 'Condensed milk', 'Pudding cups', 'Boxed stuffing mix', 'Granola bars', 'Popcorn', 'Ketchup', 'Peanut butter', 'Dried fruit', 'Cooking oil', 'Oatmeal', 'Spices', 'Cereal', 'Canned vegetables', 'Baking soda', 'Sugar', 'Pancake mix and syrup', '

<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

['Mayonnaise', 'Condensed milk', 'Boxed stuffing mix', 'Granola bars', 'Pasta', 'Canned chili', 'Popcorn', 'Crackers', 'Evaporated milk', 'Dried fruit', 'Canned beans', 'Cooking oil', 'Jelly or jam', 'Sugar', 'Flour', 'Instant mashed potatoes', 'Baking powder', 'Mustard', 'Vinegar', 'Chips', 'Trail mix', 'Macaroni and cheese', 'Canned soup', 'Shelf-stable milk', 'Pretzels', 'Tomato sauce', 'Rice', 'Condiments', 'Hot cocoa mix', 'Salad dressing']
['Mayonnaise', 'Fruit cups', 'Boxed stuffing mix', 'Granola bars', 'Pasta', 'Popcorn', 'Crackers', 'Vanilla extract', 'Cooking oil', 'Jelly or jam', 'Cereal', 'Canned vegetables', 'Boxed potatoes', 'Canned tuna or chicken', 'Baking soda', 'Salad dressing', 'Pancake mix and syrup', 'Jello cups', 'Baking powder', 'Mustard', 'Vinegar', 'Macaroni and cheese', 'Canned spaghetti or ravioli', 'Tomato sauce', 'Condiments', 'Canned fruits']
['Condensed milk', 'Boxed stuffing mix', 'Pasta', 'Tea bags', 'Crackers', 'Peanut butter', 'Vanilla extract', 'Coo

<ipython-input-254-e5cfe4068f75>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append({"PantryID": pantry_id, "TopItems": top_items_str}, ignore_index=True)
